In [79]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import pandas as pd
from pandas import ExcelWriter
from collections import Counter
import math
print("Initializing drivers ... WS")
ws_driver = CkipWordSegmenter(level=2)
print("Initializing drivers ... POS")
pos_driver = CkipPosTagger(level=2)
print("Initializing drivers ... NER")
ner_driver = CkipNerChunker(level=2)
print("Initializing drivers ... done")
print()
def init(fileName):
    global ws_driver,pos_driver,ner_driver,result,filename

    filename = "./_target/"+fileName+".xls"
    result="./_target/"+fileName+"reslut.xls"
    file = pd.read_excel(filename)

    #//print(file.head(1))
    data_with_index = file.set_index("公開/公告號")
    for i in data_with_index.index:
        if('A' in i):
            data_with_index=data_with_index.drop(i)
    #print(data_with_index.head(1))
    # * filter of the 公開專利 -> the 公開 one is pretty useless
    file =data_with_index
    print(fileName+str(len(file)))
    return file

Initializing drivers ... WS
Initializing drivers ... POS
Initializing drivers ... NER
Initializing drivers ... done



In [80]:
def run_file (file):
    var=file["摘要"]
    name=file["專利名稱"]
    global sort_key,sort_words,sent_cnt,TF_IDF
    sort_key=[]
    sort_words=[]
    sent_cnt=[]
    TF_IDF=[]
    for i in range (len(file)) :
        text =[var[i]]
        self_list=[]## get combine word 
        common_list=[]## get the most common word
        sent_cnt.append(0)
        # * Run pipeline
        # //print("Running pipeline ... WS")
        ws = ws_driver(text)
        # //print("Running pipeline ... POS")
        pos = pos_driver(ws)
        # // print("Running pipeline ... NER")
        
        ner = ner_driver(text)
        
        # //print("Running pipeline ... done")
        
        print()
        def pack_ws_pos_sentece(sentence_ws, sentence_pos):
            assert len(sentence_ws) == len(sentence_pos)
            res = []
            #TODO 字詞的組合 根據屬性表將名詞-> 專有名詞 來增加字詞的特殊性
            pre_pos=''
            pre_ws=''
            
            for word_ws, word_pos in zip(sentence_ws, sentence_pos):
                #! 寫的好爛... 可以透過語言結構做優化
                res.append(f"{word_ws}({word_pos})")
                
                if(word_pos=="Na" or "V" in word_pos):
                    common_list.append(word_ws)
                    
                if((word_pos == "Na"or word_pos=="VC") and pre_pos!='' ):
                # * use 
                    Nb=f"{pre_ws}{word_ws}"
                    # // print(Nb)
                    self_list.append(pre_ws+word_ws)
                    # //res.append(f"{Nb}(Nb)")
                    pre_pos=""
                    pre_ws=""
                        
                else :
                    #* memorizing
                    if(word_pos=="VC" or word_pos=="VD" or word_pos=="Na" or word_pos=="Nb"):
                        pre_pos=word_pos
                        pre_ws=word_ws
                        
                    else :
                        pre_pos=''
                        pre_ws=''
                        # //res.append(f"{word_ws}({word_pos})")
                if(word_pos=="PERIODCATEGORY" or word_pos=="SEMICOLONCATEGORY"):
                    sent_cnt[i]+=1
                    # //print("add "+str(sent_cnt[i]))

        # *zip指標的標記
        for sentence, sentence_ws, sentence_pos, sentence_ner in zip(text, ws, pos, ner):
            # //print(pos)
            pack_ws_pos_sentece(sentence_ws, sentence_pos)
        sort_key.append(Counter(self_list).most_common(20))
        sort_words.append(Counter(common_list).most_common(20))
        print(name[i])
        # print("done")
        # # //print(sentence)
        # print("------------")
        
        # print(sort_key[i])
        # print("------------")
        # print(sort_words[i])
        
        temp=[]
        # print(sort_key[i])
        for words in sort_key[i]:
            _TF=((words[1])/len(self_list))
            _IDF=math.log(sent_cnt[i]/words[1])

            # print(_IDF)
            # print(_TF)
            temp_num=float("{:.2f}".format(_TF*_IDF))
            temp.append((words[0],temp_num))
        temp.sort(reverse=True)    
        TF_IDF.append(temp)
        # print(TF_IDF)

        # _TF_IDF=float("{:.2f}".format(_TF*_IDF))
        # *for entity in sentence_ner:
        #//     print(entity)
        print()
        del text
        del self_list,common_list 
    return 

In [81]:
def file_done(file):
    #* date format
    ori_time=file["申請日"]
    date=[]
    for i in ori_time:
        s=str(i)
        date.append(s[0:4]+'/'+s[4:6]+'/'+s[6:8])
        #//(datetime(year=int(), month=int(s[4:6]), day=int(s[6:8])))
    #//print((date))

    #* export back to the file 
    print(len(TF_IDF))

    bank ={
            # "ID":file.index,
            "申請日":date,
            "名稱":file["專利名稱"],
            "申請人":file["申請人"],
            "IPC":file["IPC"],
            "引用專利":file["引用專利"],
            "被參考次數":file["被參考次數"],    
            "專利摘要":file["摘要"],
            "摘要關鍵字":sort_key,
            "摘要常用字":sort_words,
            "TF_IDF":TF_IDF,
            }
    # //print (pd.DataFrame(bank))
    pd.DataFrame(bank).drop_duplicates(subset ="名稱",keep = False, inplace = True)
    pd.DataFrame(bank).to_excel(result)
    return 
#// bank ={"名稱":name,
#//         "KEY":sort_key,
#//         "WORD":sort_words
#//         }
#// print (pd.DataFrame(bank))

In [83]:

#for i in ["中信金控","兆豐金控","台新金控","台灣金融控股","國泰金控","第一金控","玉山金控","土地銀行","富邦金控","開發金"]:
for i in ["玉山金控","土地銀行","富邦金控"]:
    file=init(i)
    run_file(file)
    file_done(file)
    print(i+" is done")

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

74


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Running pipeline ... done

語音通訊管理系統



Inference:   0%|          | 0/1 [00:00<?, ?it/s]

...

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Running pipeline ... done

行動理財秘書系統Mobile financial management secretary system



Inference: 100%|██████████| 1/1 [00:00<00:00,  1.85it/s]

Running pipeline ... done

雲端智能交易系統Cloud smart trading system

118
                    申請日                                                 名稱  \
公開/公告號                                                                       
TWM614130    2021/04/06  推薦名單產製系統SYSTEMS FOR GENERATING RECOMMENDATION ...   
TWM614128    2021/04/01  適於支付保險賠款的電子裝置ELECTRONIC DEVICE SUITABLE FOR PA...   
TWM614127    2021/04/01  適於分派理賠案件的電子裝置ELECTRONIC DEVICE SUITABLE FOR AS...   
TWM613667    2021/03/09   職業代碼推薦系統NUMERICAL JOB CODE RECOMMENDATION SYSTEM   
TW202119399  2019/11/08  基於區塊鏈之保險系統及其保險方法BLOCKCHAIN BASED INSURANCE SYS...   
...                 ...                                                ...   
TWM579353U   2019/03/13                    股票健診系統Stock health check system   
TWM573039U   2018/11/13  智能推播服務系統與裝置SMART PUSHING SERVICE SYSTEM AND DE...   
TWM547715U   2017/05/08      線上機器人客服系統ONLINE ROBOT CUSTOMER SERVICE SYSTEM   
TWM540337U   2016/12/28  行動理財秘書系統Mobile financial management secretary ... 


<ipython-input-81-48a7f93a7c58>:29: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  pd.DataFrame(bank).to_excel(result)
